<a href="https://colab.research.google.com/github/JRCon1/CapstoneProject/blob/main/Independent_Study(Spring_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install empyrial Riskfolio-Lib quantstats

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import empyrical as ep
import quantstats as qs
import riskfolio as rp
import datetime as dt

from empyrial import Engine, empyrial
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import expected_returns

In [ ]:
#Define the Set of Tickers, Time Interval, and Returns (First set is for training, second is testing)

#Define Tickers & Weights
tickers = ['VOO', 'QQQ', 'IWM', '^DJI']
weights = [0.25, 0.25, 0.25, 0.25]

#Define Returns Data Frames
returns = pd.DataFrame()
returns2 = pd.DataFrame()

#Define Timit Limits for Portfolio
start = dt.date(2014,1,1)
end = dt.date(2018,12,31)
#Second Portfolio
start2 = dt.date(2019,1,1)
end2 = dt.date(2023,12,31)

#Download Data
for ticker in tickers:
    returns[ticker] = yf.download(ticker, start=start, end=end)['Adj Close'].pct_change()
for ticker in tickers:
    returns2[ticker] = yf.download(ticker, start=start2, end=end2)['Adj Close'].pct_change()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
#Equally Weighted Portfolio
EW_Returns = returns2.dot(weights)
EW_Returns.name = 'Equal Weight Portfolio'
sp500 = yf.download('SPY', start=start2, end=end2)['Adj Close'].pct_change()
sp500.name = 'Benchmark (SP 500)'
qs.reports.html(EW_Returns, benchmark=sp500, output='EqualWeight.html')

[*********************100%***********************]  1 of 1 completed


In [ ]:
#Max Sharpe Portfolio
df = yf.download(tickers, start = start, end = end)['Adj Close']
mu = mean_historical_return(df, frequency=252)
S = CovarianceShrinkage(df).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.35))
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

numeric_values = [value for _, value in cleaned_weights.items()]
Sharpe_Returns = returns2.dot(numeric_values)
Sharpe_Returns.name = 'Sharpe Portfolio'
qs.reports.html(Sharpe_Returns, benchmark=sp500, output='Sharpe_Portfolio.html')

[*********************100%***********************]  4 of 4 completed


In [ ]:
ef2 = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.35))
weights2 = ef2.efficient_return(0.095057)
cleaned_weights2 = ef2.clean_weights()
cleaned_weights2

numeric_values2 = [value for _, value in cleaned_weights2.items()]
Efficient_Returns = returns2.dot(numeric_values2)
Efficient_Returns.name = 'Efficient Frontier Portfolio'
qs.reports.html(Efficient_Returns, benchmark=sp500, output='Efficient_Portfolio.html')

In [ ]:
ef3 = EfficientFrontier(mu, S, weight_bounds=(0.05, 0.35))
ef3.max_quadratic_utility()
weights3 = ef3.max_quadratic_utility()
cleaned_weights3 = ef3.clean_weights()

numeric_values3 = [value for _, value in cleaned_weights3.items()]
Max_Returns = returns2.dot(numeric_values3)
Max_Returns.name = 'Max Quadratic Utility Portfolio'
qs.reports.html(Max_Returns, benchmark=sp500, output='Max_Quad_Utility.html')